### This notebook demonstrates the generation of model cards for downstream models trained on existing Hugging Face models.

In [1]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification,AutoTokenizer, AutoModelForSequenceClassification
import torch

/Users/agamage/Desktop/Env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# pip install transformers

In [3]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

text = "I really dislike this item."

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Move inputs to the correct device
inputs = {key: value.to(device) for key, value in inputs.items()}

# Get model predictions
with torch.no_grad():
    outputs = model(**inputs)

In [5]:
# Get the predicted class (0 for negative, 1 for positive)
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

# Display the result
sentiment = "positive" if predicted_class == 1 else "negative"
print(f"The sentiment is: {sentiment}")

The sentiment is: negative


In [6]:
model.framework

'pt'

In [7]:
from patra_toolkit import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
mc = ModelCard(
    name="Sentiment Analysis Model using Hugging Face",
    version="0.1",
    short_description="Sentiment analysis model based on Hugging Face's DistilBERT fine-tuned on SST-2 dataset.",
    full_description="This model utilizes the Hugging Face Transformers framework with the DistilBERT model, fine-tuned for sentiment analysis on the SST-2 dataset. It is capable of classifying text as either positive or negative sentiment.",
    keywords="sentiment analysis, hugging face, transformers, distilbert, patra",
    author="Isuru Gamage",
    input_type="Text",
    category="classification",
    foundational_model="None",
)

mc.input_data = 'https://huggingface.co/datasets/sst2'
mc.output_data = 'https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english'

In [9]:
ai_model = AIModel(
    name="Sentiment Analysis Hugging Face Model",
    version="1.0",
    description="Sentiment analysis model using Hugging Face Transformers pre-trained DistilBERT model for positive and negative sentiment classification.",
    owner="Isuru Gamage",
    location="https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english", 
    license="Apache-2.0", 
    framework="pytorch",
    model_type="other",  
    test_accuracy=0.5
)


ai_model.populate_model_structure(model)

In [10]:
mc.ai_model = ai_model

In [11]:
mc.populate_requirements()

In [12]:
print(mc)

{
    "name": "Sentiment Analysis Model using Hugging Face",
    "version": "0.1",
    "short_description": "Sentiment analysis model based on Hugging Face's DistilBERT fine-tuned on SST-2 dataset.",
    "full_description": "This model utilizes the Hugging Face Transformers framework with the DistilBERT model, fine-tuned for sentiment analysis on the SST-2 dataset. It is capable of classifying text as either positive or negative sentiment.",
    "keywords": "sentiment analysis, hugging face, transformers, distilbert, patra",
    "author": "Isuru Gamage",
    "input_type": "Text",
    "category": "classification",
    "input_data": "https://huggingface.co/datasets/sst2",
    "output_data": "https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english",
    "foundational_model": "None",
    "ai_model": {
        "name": "Sentiment Analysis Hugging Face Model",
        "version": "1.0",
        "description": "Sentiment analysis model using Hugging Face Transformers pre-trained 

In [13]:
mc.validate()

True

In [14]:
mc.submit_model("http://127.0.0.1:5002/upload_mc")

{'message': 'Model card already exists',
 'model_card_id': '3042e494-07f3-4d53-a517-35c9a860f00d'}

In [15]:
response = mc.submit_model("http://127.0.0.1:5002/upload_mc")

In [16]:
model_card_id = response['model_card_id']

In [17]:
model_card_id = model_card_id.strip("'")

print(model_card_id)

3042e494-07f3-4d53-a517-35c9a860f00d


In [20]:
# pip install tf-keras

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.7 MB/s eta 0:00:00a 0:00:01
Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.2
    Uninstalling protobuf-5.28.2:
      Successfully uninstalled protobuf-5.28.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.67.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Retrain the HaggingFace Model

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset

In [22]:

df = pd.read_csv("/Users/agamage/Desktop/D2I/Codes/fork/patra-toolkit/examples/notebooks/data/sentiment/news_sentiment_analysis.csv")  # Adjust the path as necessary

df = df.drop(columns=['URL', 'Source','Author'])
# Combine title and description as features
df['Text'] = df['Title'] + " " + df['Description']

label_encoder_type = LabelEncoder()
label_encoder_date = LabelEncoder()

df['Type'] = label_encoder_type.fit_transform(df['Type'])

df['Published_day'] = pd.to_datetime(df['Published At']).dt.day_name()
df['Published At'] = label_encoder_date.fit_transform(df['Published_day'])

df = df.drop(columns=['Title', 'Description', 'Published_day'])

# Map sentiment labels from -1 (negative) to 0, and keep 1 as positive
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == "positive" else 0)

# Split the data into features (X) and labels (y)
X = df.drop(columns=['Sentiment'])
y = df['Sentiment'] # Target label

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)



device = torch.device("mps") if torch.has_mps else (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
model.to(device)

# Tokenize the training and test data
train_encodings = tokenizer(X_train['Text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(X_test['Text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")

# Move input tensors to the correct device
train_encodings = {key: val.to(device) for key, val in train_encodings.items()}
test_encodings = {key: val.to(device) for key, val in test_encodings.items()}

# Convert labels to tensors
y_train_tensor = torch.tensor(y_train.tolist()).to(device)
y_test_tensor = torch.tensor(y_test.tolist()).to(device)


/var/folders/hz/704s19kd1zg3phgs0khv5df00000gp/T/ipykernel_97619/2222344923.py:9: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps") if torch.has_mps else (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))


In [24]:
# Create a custom Dataset class

class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewsDataset(train_encodings, y_train_tensor)
test_dataset = ReviewsDataset(test_encodings, y_test_tensor)

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    evaluation_strategy="epoch",      # Evaluate at the end of each epoch
    logging_dir='./logs',            # Directory for storing logs
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()


evaluation_results = trainer.evaluate() 

print(evaluation_results)

model.eval()

/Users/agamage/Desktop/Env/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.410094
2,No log,0.427357
3,0.501400,0.459216


{'eval_loss': 0.45921558141708374, 'eval_runtime': 7.6455, 'eval_samples_per_second': 91.557, 'eval_steps_per_second': 11.51, 'epoch': 3.0}


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [37]:
# pip install 'accelerate>={ACCELERATE_MIN_VERSION}'

In [25]:
with torch.no_grad():
    test_outputs = model(**test_encodings)

In [26]:

device = torch.device("mps") if torch.has_mps else torch.device("cpu")

new_reviews = ["This product is amazing!", "I really dislike this item."]

# Tokenize the new reviews
encodings = tokenizer(new_reviews, truncation=True, padding=True, max_length=512, return_tensors="pt")

encodings = {key: val.to(device) for key, val in encodings.items()}

model = model.to(device)

# predictions
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**encodings)

# predicted class (0 for negative, 1 for positive)
predictions = torch.argmax(outputs.logits, dim=-1)

for review, prediction in zip(new_reviews, predictions):
    sentiment = "Positive" if prediction == 1 else "Negative"
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")

/var/folders/hz/704s19kd1zg3phgs0khv5df00000gp/T/ipykernel_97619/4040034712.py:1: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps") if torch.has_mps else torch.device("cpu")


Review: This product is amazing!
Predicted Sentiment: Positive

Review: I really dislike this item.
Predicted Sentiment: Negative



In [27]:
from patra_toolkit.patra_model_card import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis

In [28]:
mct = ModelCard(
    name="Retrain Sentiment Analysis Model using Hugging Face and News dataset",
    version="0.1",
    short_description="Retrain Sentiment analysis model based on Hugging Face's DistilBERT fine-tuned on News dataset.",
    full_description="This model utilizes the Hugging Face Transformers framework with the DistilBERT model, fine-tuned for sentiment analysis on the News dataset. It is capable of classifying text as either positive or negative sentiment.",
    keywords="sentiment analysis, hugging face, transformers, distilbert, patra, retrain",
    author="Isuru Gamage",
    input_type="Text",
    category="classification",
    foundational_model=model_card_id,
)

# Link to input and output data (in the context of Hugging Face's pre-trained model and dataset)
mct.input_data = 'https://huggingface.co/datasets/news'
mct.output_data = 'https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english'

In [29]:
ai_model = AIModel(
    name="Retrain Sentiment Analysis Hugging Face Model",
    version="1.0",
    description="Retrain Sentiment analysis model using Hugging Face Transformers pre-trained DistilBERT model for positive and negative sentiment classification.",
    owner="Isuru Gamage",
    location="https://huggingface.co/distilbert-base-uncased-finetuned-news-english/train", 
    license="Apache-2.0", 
    framework="pytorch",
    model_type="other",  
    test_accuracy=0  
)


ai_model.populate_model_structure(model)

In [30]:
ai_model.add_metric("Test loss", 0.22)
ai_model.add_metric("Epochs", 100)
ai_model.add_metric("Batch Size", 32)
ai_model.add_metric("Optimizer", "Adam")
ai_model.add_metric("Learning Rate", 0.0001)
ai_model.add_metric("Input Shape", "(26048, 100)")

In [31]:
mct.ai_model = ai_model

In [32]:
mct.populate_requirements()

In [33]:
from torch.nn.functional import softmax
import numpy as np

y_pred_prob = softmax(test_outputs.logits, dim=1).cpu().numpy()

print(f"Probabilities for the first 5 samples: \n{y_pred_prob[:5]}")

# If you want to convert these probabilities into binary predictions:
y_pred = np.argmax(y_pred_prob, axis=1)

Probabilities for the first 5 samples: 
[[0.00149967 0.99850035]
 [0.00265727 0.99734277]
 [0.9969002  0.00309979]
 [0.00168173 0.9983183 ]
 [0.00421658 0.99578345]]


In [34]:
print(mct)

{
    "name": "Retrain Sentiment Analysis Model using Hugging Face and News dataset",
    "version": "0.1",
    "short_description": "Retrain Sentiment analysis model based on Hugging Face's DistilBERT fine-tuned on News dataset.",
    "full_description": "This model utilizes the Hugging Face Transformers framework with the DistilBERT model, fine-tuned for sentiment analysis on the News dataset. It is capable of classifying text as either positive or negative sentiment.",
    "keywords": "sentiment analysis, hugging face, transformers, distilbert, patra, retrain",
    "author": "Isuru Gamage",
    "input_type": "Text",
    "category": "classification",
    "input_data": "https://huggingface.co/datasets/news",
    "output_data": "https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english",
    "foundational_model": "3042e494-07f3-4d53-a517-35c9a860f00d",
    "ai_model": {
        "name": "Retrain Sentiment Analysis Hugging Face Model",
        "version": "1.0",
        "desc

In [35]:
mct.validate()

True

In [36]:
mct.submit("http://127.0.0.1:5002")

{'message': 'Successfully uploaded the model card',
 'model_card_id': 'bd9c3eec-cbfb-42f2-9962-e68ce5b95ad0'}